TODOS:<br>

- Übersicht über alle Verfahren geben in kapitel 3 (Tabelle?!)
- überprüfen Formulierung
- untertitel der grafik bei LSA in pdf überprüfen

<h1>Inhaltsverzeichnis<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Einführung" data-toc-modified-id="Einführung-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Einführung</a></span><ul class="toc-item"><li><span><a href="#Das-Bag-of-Words-Modell" data-toc-modified-id="Das-Bag-of-Words-Modell-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Das Bag-of-Words Modell</a></span></li><li><span><a href="#Grenzen-des-Bag-of-Words-Modells" data-toc-modified-id="Grenzen-des-Bag-of-Words-Modells-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Grenzen des Bag-of-Words Modells</a></span></li></ul></li><li><span><a href="#LSA" data-toc-modified-id="LSA-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>LSA</a></span></li><li><span><a href="#Word-Embeddings" data-toc-modified-id="Word-Embeddings-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Word Embeddings</a></span><ul class="toc-item"><li><span><a href="#Word2Vec" data-toc-modified-id="Word2Vec-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Word2Vec</a></span><ul class="toc-item"><li><span><a href="#Vortrainierte-Embeddings-verwenden" data-toc-modified-id="Vortrainierte-Embeddings-verwenden-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Vortrainierte Embeddings verwenden</a></span></li><li><span><a href="#Eigene-Embeddings-verwenden" data-toc-modified-id="Eigene-Embeddings-verwenden-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Eigene Embeddings verwenden</a></span></li></ul></li><li><span><a href="#Glove" data-toc-modified-id="Glove-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Glove</a></span></li><li><span><a href="#FastText" data-toc-modified-id="FastText-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>FastText</a></span></li><li><span><a href="#ELMo" data-toc-modified-id="ELMo-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>ELMo</a></span></li><li><span><a href="#BERT" data-toc-modified-id="BERT-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>BERT</a></span></li></ul></li></ul></div>

In [52]:
from gensim.models import Word2Vec
import gensim.downloader as gensim_downloader

from nltk import word_tokenize
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import urllib.request

from utils import preprocess_text, plot_word_embeddings

ImportError: cannot import name 'plot_word_embeddings' from 'utils' (/Users/jan/Desktop/informatik_programme/wordembeddings/app/utils.py)

In [23]:
text_w2v = ["das kind sagt, dass es feuerwehrmann sein möchte",
            "der erwachsene sagt, dass er lieber kein feuerwehrmann sein möchte",
            "das kind fragt, was der erwachsene denn lieber sein möchte",
            "der erwachsene sagt, dass er lieber polizist sein möchte"]
#text_w2v = [word_tokenize(sen) for sen in text_w2v]
df = pd.DataFrame(text_w2v, columns = ["text"])

In [ ]:
%%time
corpus = pd.read_csv("../corpora/small_amazon_reviews_electronic.csv")
corpus["review"] = corpus.review.apply(lambda x: preprocess_text(x))
texts = [word_tokenize(row["review"]) for idx, row in corpus.iterrows()]

In [ ]:
url = 'http://cloud.devmount.de/d2bc5672c523b086/german.model'
urllib.request.urlretrieve(url, "german_model.bin")

# Einführung

Das **Natural Language Processing** (kurz: NLP) befasst sich mit Methoden und Verfahren zur maschinellen Verarbeitung von natürlicher Sprache in Form von Worten, Texten oder ganzen Korpora. Bevor jedoch NLP Verfahren wie die Textklassifikation oder das Topic Modelling auf die Textdaten angewendet werden können, müssen diese in eine Darstellungsweise umgewandelt werden, mit der die Verfahren arbeiten können. Die rohen Textdaten werden daher in **Vektoren**, die aus Zahlen bestehen, umgewandelt. Dieser Vorgang nennt sich **Vektorisierung**. Ein Wort wie "Baum" kann dadurch als Vektor aufgefasst werden. Natürlich können auch andere Features aus den Texten als Vektoren dargestellt werden; so ist es auch möglich, einzelne Buchstaben, Phrasen, Sätze, Segmente oder ganze Texte als Features aus den Textdaten zu extrahieren und diese zu vektorisieren. In der folgenden Übersicht werden jedoch vorwiegend Wörter als Features verwendet.

## Das Bag-of-Words Modell

Das wohl einfachste Verfahren zur Darstellung von Wörtern als Vektoren ist das **Bag-of-Words** Modell. Wörter werden hier als eindimensionale Vektoren (= einfache Zahl) dargestellt, wobei jedes individuelle Wort einen individuellen eindimensionalen Vektor (auch: **Index**) zugeordnet bekommt. Die Zuordnungen jedes einzigartigen Wortes zu seinem Vektor werden in einem *Vokabular* gespeichert. Nun können mithilfe dieses Vokabulars auch ganze Sätze oder sogar Texte dargestellt werden. Dafür wird für jeden Satz/Text ein Vektor gebildet, der die gleiche Länge wie das Vokabular hat. Jedem Eintrag des Vektors wird anhand des Vokabulars ein Wort zugeordnet. Der Satz/Text wird dann als Vektor aus **absoluten Termhäufigkeiten** dargestellt, wo an jeder Stelle, an dem ein Wort aus dem Vokabular in dem Text vorkommt, die Häufigkeit des Wortes in dem jeweiligen Satz/Text steht und an jeder anderen Stelle eine $0$, da es kein einziges Mal vorkommt.[<sup>1</sup>](#fn1) Dies soll im Folgenden anhand eines Code-Beispiels erläutert werden. Zuerst wird das Vokabular aller Texte dargestellt, bei dem die Wörter einem Index zugeordnet werden (es wird ab $0$ gezählt). Danach werden die vektorisierten Sätze/Texte angezeigt.

<hr style="border: 0.1px solid black;"/>
<div id="fn1" style="font-size:8pt; line-height:1; padding-left: 1em; text-indent: -1em"><sup style="font-size:5pt">1</sup> &nbsp; Dies ist nur eine Möglichkeit, die Häufigkeit eines Wortes beim Bag-of-Words Modell darzustellen. Eine weitere Möglichkeit wären <b>binäre Häufigkeiten</b>, bei denen das Vorkommen eines Wortes mit einer $1$ und die Abwesenheit eines Wortes mit einer $0$ gekennzeichnet werden. Um häufigen Wörtern in den Dokumenten weniger Gewicht zu geben, da dies oft einen geringeren Informationsgehalt besitzen, ist es auch möglich, das Bag-of-Words Modell in der Kombination mit dem <b>TF-IDF Maß</b> aus dem Bereich des Information Retrievals zu verwenden, bei dem die Häufigkeit von Worten skaliert wird.</div> 

In [2]:
text = ["ich gehe nachher zur bank, um etwas geld zu holen",
        "ich möchte mich kurz auf die bank setzen",
        "um mir etwas zu essen zu holen, stand ich von der bank auf", 
        "auf der hölzernen bank neben der bank liegt noch geld"]

vectorizer = CountVectorizer()
vector = vectorizer.fit_transform(text)
print(vectorizer.vocabulary_)

{'ich': 10, 'gehe': 6, 'nachher': 16, 'zur': 24, 'bank': 1, 'um': 21, 'etwas': 5, 'geld': 7, 'zu': 23, 'holen': 8, 'möchte': 15, 'mich': 13, 'kurz': 11, 'auf': 0, 'die': 3, 'setzen': 19, 'mir': 14, 'essen': 4, 'stand': 20, 'von': 22, 'der': 2, 'hölzernen': 9, 'neben': 17, 'liegt': 12, 'noch': 18}


In [3]:
print(vector.toarray())

[[0 1 0 0 0 1 1 1 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 1]
 [1 1 0 1 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 0 0 0 0 0]
 [1 1 1 0 1 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 1 1 2 0]
 [1 2 2 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0]]


## Grenzen des Bag-of-Words Modells

Aufgrund seiner Einfachheit ist das Bag-of-Words Modell gut verständlich und sehr schnell umsetzbar. Es hat jedoch eine Reihe an Nachteilen, von deinen einige im Folgenden kurz erläutert werden:

- **Keine Informationen über Reihenfolge der Wörter.** Beim Bag-of-Words Modell wird jegliche Information über die Reihenfolge der Wörter verworfen, der Kontext eines Wortes bleibt unberücksichtigt. Dies wird auch durch den Namen dieses Modells deutlich: Die Bezeichnung "bag" (deutsch: Sack) soll darauf hinweisen, dass alle Informationen über die Struktur oder Reihenfolge der Wörter im Dokument verworfen werden, da sie metaphorisch in einen "Sack" geworfen werden. Die Reihenfolge lässt sich auch nicht im Nachhinein rekonstruieren. Insgesamt gehen somit sehr viele semantische Informationen verloren. Eine Lösung, bei der die Reihenfolge der Worte berücksichtigt werden kann, ist die Verwendung von **N-Grammen** oder **LSA** (siehe Kapitel 2 TODO). 
- **Spärlichkeit von Wortvektoren.** Umso mehr verschiedene Worte in den verwendeten Texten vorkommen, umso größer wird das Vokabular. Dies kann oft zu sehr spärlichen (engl. *sparse*) Wortvektoren führen. Besteht das Vokabular aus 500000 Worten, ein Text aber nur aus 50 verschiedenen Worten, sind nur 0.01% der Stellen des 500000 langen Wortvektors mit Einsen besetzt, der Rest nur mit Nullen. Dies führt dazu, dass eine große Menge an Rechenspeicher für die Verarbeitung der riesigen Matrizen benötigt wird. Weiterhin werden wenige Informationen in sehr großen Repräsentationsräumen benutzt, wodurch es für einige NLP Verfahren und Modelle problematisch ist, diese wenigen Information effizient zu nutzen. Eine Lösung bieten dichtbesetzte **Word Embeddings**, die in den nächsten Kapiteln behandelt werden.
- **Abbildung der Mehrdeutigkeit von Worten**. Wörter können trotz gleicher Schreibweise mehrere Bedeutungen haben, welche sich durch den Kontext des Wortes zeigen können. Dies wird durch das Bag-of-Words Modell nicht abgebildet. Eine mögliche Lösung wäre die Verwendung von **kontextabhängigen Word Embeddings** wie die **BERT-Embeddings** in Kapitel 7 (TODO).

# LSA

**Latent Semantic Analysis** (kurz: LSA, auch: *Latent Semantic Indexing*) ist ein Verfahren aus dem Bereich des Information Retrievals aus dem Jahre 1990. Bei diesem Verfahren werden Dokumente und Terme (repräsentiert durch eine **Term-Dokument Matrix**) in einem latenten Raum abgebildet, der aus **Konzepten** (oder **Hauptkomponenten**) besteht. Dokumente, die ähnlich zueinander sind, d.h. aus ähnlichen Konzepten bestehen, werden in diesem Raum näher beieinander platziert. Dies wird durch die folgende Grafik deutlich.


![lsa](img/lsa.png)

Grafik von Susan Dumais, siehe <a href="http://www.ifis.uni-luebeck.de/~moeller/tuhh-lectures/mmieir-sose-12/05-Latent-Semantic-Analysis.pdf">Präsentation</a>.

Das Ziel der LSA ist es, die Konzepte innerhalb der Dokumente zu finden. Dabei greift das Verfahren auf eine Technik der linearen Algebra zurück, der **Singulärwertzerlegung** (englisch: Singular Value Decomposition). Die Idee dabei ist, dass die Term-Dokument Matrix aus **Hauptdimensionen**, welche die wichtigen Konzepte der Dokumente beinhalten, und aus weniger aussagekräftigen Dimensionen mit unwichtigen Termen besteht. Mithilfe der Singulärwertzerlegung wird die originale Term-Dokument Matrix in drei Matrizen aufgeteilt, wobei die beiden äußeren Matrizen aus den linken bzw. rechten orthonormalen Eigenvektoren bestehen und die mittlere Matrix eine Diagonalmatrix ist, die die singulären Werte der Originalmatrix enthält. Mit Hilfe dieser Zerlegung kann eine **Approximation** der Originalmatrix mit einer kleiner dimensionierten Matrix erreicht werden. Die singulären Werte in der Diagonalmatrix sind nach ihrer Größe absteigend geordnet. Singulärwerte, die unter einem bestimmten Schwellenwert liegen, werden entfernt. Auch in den anderen Matrizen werden entsprechende Zeilen oder Spalten entfernt. Mit Hilfe der reduzierten Matrizen erhält man durch Matrixmultiplikation die optimale Approximation der Originalmatrix, die kleiner als die originale Term-Dokument Matrix ist, da Informationen aus den weniger aussagekräftigen Dimensionen verworfen wurden. Weiterhin werden bei der Dimensionsreduktion auch ähnliche Konzepte zusammengefasst, so werden z.B. Worte wie "Tür" und "Tor" in einem Konzept zusammengefasst.

Der Vorteil der LSA ist, dass anders als beim Bag-of-Words Modell die **Semantik** der Dokumente widergegeben werden kann. Zudem werden **Synonyme** zusammengefasst. Probleme hat LSA jedoch mit der **Polysemie**. Zudem ist der Algorithmus sehr rechenaufwendig.

TODO: mehr?

# Word Embeddings

**Word Embeddings** (deutsch: Worteinbettungen) sind die Sammelbezeichnung für eine Reihe von Sprachmodellierungstechniken. Anders als beim Bag-of-Words Modell werden Wörter mit ähnlichen Bedeutungen ähnlich dargestellt, wobei der Kontext der Wörter berücksichtigt wird. Word Embeddings repräsentieren Wörter als Vektoren in einem multidimensionalen semantischen Raum. In diesem Raum werden Wörter, die ähnlich zueinander sind, näher beieinander platziert. Diesen Raum kann man sich etwa folgendermaßen vorstellen:<br>


![wordembeddings](https://miro.medium.com/max/1291/1*Fat62b1ZITOFMPXTcHNkLw.jpeg)

Die grundsätzliche Idee von Word Embeddings basiert auf der **Distributionellen Hypothese** von  John Rupert Firth, die besagt, dass die Bedeutung eines Wortes durch sein Umfeld geprägt ist. Wörter, die einen ähnlichen Kontext besitzen, haben eine ähnliche Bedeutung. Anders als vorherige Vektorisierungsmethoden basieren Word Embeddings auf **Vorhersage-Modellen** (engl.: prediction models), indem Wörter durch Wahrscheinlichkeiten anstatt Häufigkeiten wie beim Bag-of-Words Modell oder bei der Latent Semantic Analysis dargestellt werden. Weiterhin sind die Wortvektoren von Word Embeddings anders als beim Bag-of-Words Modell **dichtbesetzt** (engl. **dense**) und haben weitaus weniger Dimensionen (100-800 Dimensionen anstatt 100000-1000000, je nach der Größe des Vokabulars). Dadurch haben die Wortvektoren eine viel geringere Größe, bieten trotzdem eine effizientere und komplexere Darstellung der Wörter.


Word Embeddings wurde ab 2013 durch die Einführung des Algorithmus **Word2vec** populär, in den Jahren darauf folgten weitere Word Embedding Algorithmen wie GloVe, FastText, ELMo und BERT. In der folgenden Tabelle sind die wichtigsten Unterschiede der verschiedenen Embeddings zusammengefasst, genauere Erläuterungen der Embeddings befinden sich in den folgenden Kapiteln.

TODO: ausfüllen & mehr, gucken wie anders dargestellt werden kann

<table>
    <th></th>
    <th>Word2Vec</th>
    <th>Glove</th>
    <th>FastText</th>
    <th>ELMo</th>
    <th>BERT</th>
    <tr>
        <td>Entstehungsjahr</td>
        <td>2013</td>
        <td>2014</td>
        <td>2016</td>
        <td>2018</td>
        <td>2018</td>
    </tr> 
    <tr>
        <td>Out of vocabulary Fehler</td>
        <td>ja</td>
        <td>ja</td>
        <td>nein</td>
        <td>nein</td>
        <td>nein</td>
    </tr> 
    <tr>
        <td>Kontextsensitiv</td>
        <td>nein</td>
        <td>nein</td>
        <td>nein</td>
        <td>ja</td>
        <td>ja</td>
    </tr> 
    <tr>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
    </tr> 
</table>

## Word2Vec

Die Popularität von Word Embeddings ist vor allem **Word2Vec** geschuldet, welches 2013 von Tomas Mikolov und weiteren Mitgliedern von Google publiziert wurde. Word2Vec ist im eigentlichen Sinne kein alleinstehender Algorithmus, sondern besteht aus zwei Techniken: dem **Continous Bag of Words** (**CBOW**) und dem **Skip-gram Model**. **Continous Bag of Words** versucht die Wahrscheinlichkeit eines Wortes oder einer Gruppe von Wörtern anhand eines gegebenen Kontext vorauszusagen:

![cbow](img/cbow.png)


Das **Skip-gram Model** funktioniert wie CBOW, nur anders herum. Das Modell versucht, anhand eines gegebenen Wortes den Kontext vorauszusagen:

![skip-gram](img/skipgram.png)

<br>Word2Vec nutzt wahlweise eine dieser Techniken, um aus rohen Textdaten mithilfe eines Neuronales Netzes Wortvektoren zu erstellen. Dabei kann Word2Vec bei vielen Implementierungen (z.b. bei Gensim) auf zwei Arten verwendet werden: Entweder werden bereits vortrainierte Embeddings geladen oder es werden eigene Embeddings auf eigenen Textdaten trainiert.

### Vortrainierte Embeddings verwenden

In [45]:
pretrained_w2v = gensim.models.KeyedVectors.load_word2vec_format('german_model.bin', 
                                                                 binary=True)

In [47]:
print("Länge des word embedding Vektors: ", len(pretrained_w2v['Koenig']))
pretrained_w2v['Koenig'][0:4]

Länge des word embedding Vektors:  300


array([0.1141477 , 0.11475459, 0.10349818, 0.29905704], dtype=float32)

In [49]:
pretrained_w2v.most_similar('Koenig', topn=12)

[('Prinz', 0.7856332659721375),
 ('Koenigs', 0.7356206774711609),
 ('Koenigin', 0.7255839109420776),
 ('Jungkoenig', 0.7053836584091187),
 ('Kaiser', 0.7047818303108215),
 ('Prinzen', 0.6925939321517944),
 ('Jungschuetzenkoenig', 0.6864457726478577),
 ('Kronprinz_Philippe', 0.6853584051132202),
 ('Koenig_Willem', 0.6849293112754822),
 ('Thron', 0.6848670840263367),
 ('Willem-_Alexander', 0.6764872074127197),
 ('Prinzessin', 0.6718595027923584)]

TODO: hier weiter
hier gucken https://github.com/fotisj/word_embeddings/blob/master/w2v_intro.ipynb

plot_word_embeddings benutzen, um graphisch zu zeigen

### Eigene Embeddings verwenden

In [33]:
%%time
word2vec_cbow = Word2Vec(texts, min_count=1, size=100, window=5, sg=0)
word2vec_skipgram = Word2Vec(texts, min_count=1, size=100, window=5, sg=1)

CPU times: user 5min 56s, sys: 3.28 s, total: 5min 59s
Wall time: 2min 39s


In [ ]:
word2vec.wv.most_similar('smartphone')

TODO: 
- Vorwissen?
- Unterschied BOW: 
    - dense
    - 

## Glove

TODO

## FastText

TODO

## ELMo

TODO

## BERT


BERT steht für "Bidirectional Encoder Representations from Transformers" und brach bei seiner Veröffentlichung 2018 eine ganze Reihe von Rekorden für Aufgaben des Natural Language Processing. 